In [1]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_MODEL = "gpt-3.5-turbo"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
class Country(BaseModel):
    capital: str = Field(description="capital of the country")
    name: str = Field(description="name of the country")

PROMPT_COUNTRY_INFO = """
    Provide information about {country}.
    {format_instructions}
    """

In [4]:
parser = PydanticOutputParser(pydantic_object=Country)
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)
message = HumanMessagePromptTemplate.from_template(
    template=PROMPT_COUNTRY_INFO,
)

/tmp/ipykernel_39503/1746828345.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)


In [5]:
chat_prompt = ChatPromptTemplate.from_messages([message])
chat_prompt

ChatPromptTemplate(input_variables=['country', 'format_instructions'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country', 'format_instructions'], input_types={}, partial_variables={}, template='\n    Provide information about {country}.\n    {format_instructions}\n    '), additional_kwargs={})])

In [6]:
country_name = "Chile"

In [7]:
print("Generating response...")
chat_prompt_with_values = chat_prompt.format_prompt(
    country=country_name, format_instructions=parser.get_format_instructions()
)

Generating response...


In [8]:
output = llm(chat_prompt_with_values.to_messages())
country = parser.parse(output.content)
print(f"The capital of {country.name} is {country.capital}.")

/tmp/ipykernel_39503/2875788004.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm(chat_prompt_with_values.to_messages())


The capital of Chile is Santiago.


# Pydantic

In [8]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field, field_validator, ConfigDict
from typing import List, Optional
from datetime import datetime

class CodeReview(BaseModel):
    score: int = Field(..., ge=1, le=10, description="Code quality score (1-10)")
    issues: List[str] = Field(..., description="List of code issues found")
    suggestions: List[str] = Field(..., description="Improvement suggestions")
    complexity: str = Field(..., description="Code complexity level")
    estimated_fix_time: str = Field(..., description="Estimated time to fix issues")

In [10]:
from langchain_core.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=CodeReview)
format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"score": {"description": "Code quality score (1-10)", "maximum": 10, "minimum": 1, "title": "Score", "type": "integer"}, "issues": {"description": "List of code issues found", "items": {"type": "string"}, "title": "Issues", "type": "array"}, "suggestions": {"description": "Improvement suggestions", "items": {"type": "string"}, "title": "Suggestions", "type": "array"}, "complexity": {"description": "Code complexity level", "title": "Complexity", "type": "string"}, "estimated_fix_time": {"description": "Estimated time to fix issu

In [11]:
example_output = {
            "score": 7,
            "issues": ["Missing error handling", "No input validation"],
            "suggestions": ["Add try-catch blocks", "Validate user input"],
            "complexity": "Medium",
            "estimated_fix_time": "2-3 hours"
        }
        
parsed_review = CodeReview(**example_output)
print(f"\n✅ Parsed review score: {parsed_review.score}")
print(f"   Issues found: {len(parsed_review.issues)}")


✅ Parsed review score: 7
   Issues found: 2


In [12]:
parsed_review.issues


['Missing error handling', 'No input validation']